In [1]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

ds = load_dataset("rajpurkar/squad")


import re
from tqdm import tqdm
import torch
from torch.optim import AdamW
import matplotlib.pyplot as plt
from torch import nn
import random

import pandas as pd


device='cuda'

size=10000
batch=4
epochs=5

model = AutoModelForCausalLM.from_pretrained("../distillLLAMA2")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
train_dataset=ds["train"].shuffle(seed=42).select(range(size))

train_data=[]
for i in range(size):
    tokenized = tokenizer(train_dataset[i]['context']+train_dataset[i]['question'] + train_dataset[i]['answers']['text'][0], padding="max_length", max_length=512, truncation=True, return_tensors="pt")
    cqlen = len(tokenizer(train_dataset[i]['context']+train_dataset[i]['question'])['input_ids'])
    length=torch.sum(tokenized['attention_mask'][0])
    input_ids = tokenized['input_ids'].squeeze().tolist()
    attention_mask = tokenized['attention_mask'].squeeze().tolist()
    
    labels=[-100]*512
    for i in range(length-cqlen):
        labels[cqlen+i-1]=input_ids[cqlen+i]
    train_data.append({"input_ids": input_ids, "labels": labels, "attention_mask":attention_mask})

In [2]:
train_data[9]

{'input_ids': [128000,
  96056,
  370,
  49112,
  645,
  5762,
  10597,
  323,
  13042,
  83476,
  304,
  10846,
  15212,
  11,
  25431,
  323,
  1063,
  559,
  13005,
  4633,
  10846,
  4435,
  27833,
  13,
  578,
  14154,
  8620,
  27458,
  54877,
  11354,
  439,
  18210,
  315,
  12646,
  4914,
  477,
  4998,
  76052,
  13,
  763,
  36684,
  89490,
  309,
  1122,
  17649,
  11,
  279,
  25706,
  33894,
  315,
  21456,
  35668,
  71,
  706,
  682,
  36807,
  311,
  507,
  15357,
  460,
  902,
  89522,
  279,
  91182,
  3225,
  315,
  4998,
  76052,
  13,
  22395,
  267,
  279,
  3765,
  4775,
  1572,
  315,
  8494,
  315,
  279,
  18925,
  261,
  24341,
  4221,
  5315,
  11,
  26828,
  61495,
  323,
  37482,
  2676,
  1099,
  16115,
  10434,
  439,
  4443,
  39562,
  311,
  91937,
  13,
  763,
  279,
  1162,
  315,
  279,
  364,
  24661,
  6,
  30773,
  89069,
  315,
  279,
  27930,
  1494,
  2850,
  11,
  433,
  374,
  279,
  52232,
  26976,
  285,
  902,
  10187,
  1790,
  13042,
 